In [1]:
import requests
import json
from pathlib import Path
from tqdm import tqdm
import time
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
apiKey = os.environ['SEMANTIC_SCHOLAR_API']

def load_json(path):
    f = open(path)
    data = json.load(f)
    f.close
    return data
    
def request_batch_metadata(ids, fields="paperId"):
    r = requests.post(
        'https://api.semanticscholar.org/graph/v1/paper/batch',
        params={'fields': fields},
        json={"ids": ids},
        headers={"x-api-key":apiKey}
    )
    return r.json()

# Initial references metadata download

In [20]:
pathlist = []
for year in ["before_2020", "2020", "2021", "2022", "2023", "2024"]:
    pathlist = pathlist + sorted(Path(f"./data/{year}").glob('**/*.json'))

with open('./data/references_metadata.json', 'r') as f:
    references = json.load(f)

for i, path in enumerate(tqdm(pathlist)):
    if ".proceedings.json" not in path.parts[-1]:
        tutorial_json = load_json(path)
        ids = [ref for key, values in tutorial_json.items() for section in values["readingList"] for ref in section["referencesIds"] if ref and ref not in references]
        if ids != []:
            res = request_batch_metadata(ids, 
                                          fields='paperId,externalIds,title,abstract,year,publicationVenue,s2FieldsOfStudy,referenceCount,citationCount,isOpenAccess')
            time.sleep(1.1)
        else:
            res = []
        for ref in res:
            if ref["paperId"] not in references:
                references[ref["paperId"]] = ref
            
with open('./data/references_metadata.json', 'w') as f:
    json.dump(references, f, indent=2)

100%|███████████████████████████████████████████| 141/141 [00:03<00:00, 43.65it/s]


# Update citation fields

In [21]:
with open('./data/references_metadata.json', 'r') as f:
    references = json.load(f)

keys = list(references.keys())
keys_chunks = [keys[i:i + 500] for i in range(0, len(keys), 500)]

for chunk in keys_chunks:
    res = request_batch_metadata(chunk, fields="paperId,citationCount")
    for metadata in res:
        if references[metadata["paperId"]]["citationCount"] != metadata["citationCount"]:
            print(f"Updated {metadata['paperId']}: {references[metadata['paperId']]['citationCount']} -> {metadata['citationCount']}" )
            references[metadata["paperId"]]["citationCount"] = metadata["citationCount"]
            
with open('./data/references_metadata.json', 'w') as f:
    json.dump(references, f, indent=2)

# Detect missing fields to fill

In [22]:
with open('./data/references_metadata.json', 'r') as f:
    references = json.load(f)

with open('./data/references_missing_metadata.json', 'r') as f:
    references_missing_metadata = json.load(f)
    
for key, ref in references.items():
    if key not in references_missing_metadata:
        if not ref["abstract"] or not ref["year"]:
            references_missing_metadata[key] = {}
        if not ref["abstract"] and "abstract" not in references_missing_metadata[key]: 
                references_missing_metadata[key]["abstract"] = None
        if not ref["year"] and "year" not in references_missing_metadata[key]:
                references_missing_metadata[key]["year"] = None

with open('./data/references_missing_metadata.json', 'w') as f:
    json.dump(references_missing_metadata, f, indent=2)